# This notebook generates a CSV of points with large prediction errors in V15 or V20

In [2]:
import pandas as pd
df = pd.read_pickle('tmp_data/clean_yearly_data.pkl')

In [17]:
df

,latitude_ERA,longitude_ERA,MODIS_LST,V15_prediction,V20_prediction,V20X_prediction,V15X_prediction,V15_error,V20_error,V20X_error,...,V15_dl,V20_dl,percentage_change_in_dl,cl_change_is_significant,si10_change_is_significant,cvh_change_is_significant,cvl_change_is_significant,lsm_change_is_significant,dl_change_is_significant,bitstring
0,-70.117048,-75.75,256.971771,260.839691,258.764496,258.308044,260.245789,5.176608,4.774697,4.901756,...,68.036133,222.733398,2.273751,False,True,False,False,True,True,000111
1,-70.117048,-75.00,256.868408,260.022491,257.034821,254.135437,262.143463,4.589613,4.524141,5.674824,...,25.000000,105.971680,3.238867,False,True,False,False,False,True,000101
2,-70.117048,-72.00,255.929306,258.782562,257.770081,256.421112,257.124664,5.802632,5.922895,5.830910,...,25.000000,10.000000,-0.600000,False,False,False,False,False,True,000100
3,-70.117048,-71.25,254.781509,258.479736,257.310944,257.045654,257.465698,5.732193,5.754484,5.606140,...,25.000000,10.000000,-0.600000,False,False,False,False,False,True,000100
4,-70.117048,-70.50,255.270538,258.035889,258.086243,256.977478,259.335144,6.000848,5.888017,6.004581,...,25.000000,10.000000,-0.600000,False,True,False,False,False,True,000101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149805,70.117048,168.00,264.279388,264.181641,264.485321,264.104553,261.404938,3.836673,3.708954,3.456014,...,30.217773,31.002930,0.025983,False,False,False,False,False,False,000000
149806,70.117048,168.75,263.243958,263.457367,263.287903,263.835266,260.608521,3.779803,3.540926,3.669104,...,19.599609,18.327148,-0.064923,False,False,False,False,False,False,000000
149807,70.117048,170.25,266.316162,262.494171,262.690735,262.242371,264.249634,5.866361,5.818576,6.012778,...,20.901367,20.796875,-0.004999,False,False,False,False,False,False,000000
149808,70.117048,171.00,266.383087,264.283325,264.261658,264.064941,263.495117,5.195374,5.332411,5.318110,...,28.710938,27.846680,-0.030102,False,False,False,False,False,False,000000


In [29]:
#Get the points where the V15 error is large
V15error = df.query('V15_error > 5.1')


#Get the points where the V20 error is large
V20error = df.query('V20_error > 5.1')


#Save both
V15error.to_csv('tmp_data/V15_error_GT_5P1K.csv')
V20error.to_csv('tmp_data/V20_error_GT_5P1K.csv')



----

# Generate CSV of ALL points WITH orography

In [36]:
df = pd.read_pickle('tmp_data/clean_yearly_data.pkl')

In [63]:
selected_columns = ['latitude_ERA', 'longitude_ERA', 'MODIS_LST', 
 'V15_prediction','V20_prediction', 'V20X_prediction', 
'V15_error', 'V20_error', 'V20X_error', 
'V15_cl', 'V20_cl',
'V15_si10', 'V20_si10', 
'V15_cvh', 'V20_cvh', 
'V15_cvl', 'V20_cvl', 
'V15_lsm', 'V20_lsm', 
'V15_dl', 'V20_dl', 
'bitstring']
dflite = df[selected_columns].copy()

In [51]:
import xarray as xr
def generate_oro_df(path):
    
    z15 = xr.open_dataset(path,engine='cfgrib',backend_kwargs={'indexpath': ''}) 
    z15 =  z15.assign_coords({"longitude": (((z15.longitude + 180) % 360) - 180)}) #long3 ---> long1
    z15 = z15.to_dataframe().reset_index()[['latitude','longitude','z']].copy()
    
    
    return z15

In [52]:
#Paths to orography files
oroV15 = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/climate.v015/climate.v015/639l_2/lsmoro'
oroV20 = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/climate.v020/climate.v020/639l_2/lsmoro'

In [53]:
#Generate a pandas df of the oro
z15 = generate_oro_df(oroV15)
z20 = generate_oro_df(oroV20)



In [62]:
#Rename the geopoential column
z15 = z15.rename(columns={"z": "V15_z"})
z20 = z20.rename(columns={"z": "V20_z"})


In [68]:
z = pd.merge(z15,z20,on=['latitude','longitude'])

,latitude,longitude,V15_z,V20_z
0,89.784877,0.0,5.503784,5.220947
1,89.784877,20.0,6.609253,6.342041
2,89.784877,40.0,5.078003,4.869385
3,89.784877,60.0,2.906128,2.783447
4,89.784877,80.0,1.370972,1.330322
...,...,...,...,...
542075,-89.784877,-100.0,27070.917969,27070.994141
542076,-89.784877,-80.0,27001.785156,27001.732422
542077,-89.784877,-60.0,26972.625000,26972.462891
542078,-89.784877,-40.0,26996.261719,26996.072266


In [70]:
dfout = pd.merge(dflite,z,left_on=['latitude_ERA','longitude_ERA'],right_on=['latitude','longitude'],how = 'left')

In [71]:
dfout

,latitude_ERA,longitude_ERA,MODIS_LST,V15_prediction,V20_prediction,V20X_prediction,V15_error,V20_error,V20X_error,V15_cl,...,V20_cvl,V15_lsm,V20_lsm,V15_dl,V20_dl,bitstring,latitude,longitude,V15_z,V20_z
0,-70.117048,-75.75,256.971771,260.839691,258.764496,258.308044,5.176608,4.774697,4.901756,0.0,...,0.0,0.448748,0.315605,68.036133,222.733398,000111,-70.117048,-75.75,756.718628,773.924072
1,-70.117048,-75.00,256.868408,260.022491,257.034821,254.135437,4.589613,4.524141,5.674824,0.0,...,0.0,0.784160,0.708793,25.000000,105.971680,000101,-70.117048,-75.00,1161.531128,1167.908447
2,-70.117048,-72.00,255.929306,258.782562,257.770081,256.421112,5.802632,5.922895,5.830910,0.0,...,0.0,1.000000,1.000000,25.000000,10.000000,000100,-70.117048,-72.00,750.277222,741.451416
3,-70.117048,-71.25,254.781509,258.479736,257.310944,257.045654,5.732193,5.754484,5.606140,0.0,...,0.0,0.998763,1.000000,25.000000,10.000000,000100,-70.117048,-71.25,187.328003,180.619385
4,-70.117048,-70.50,255.270538,258.035889,258.086243,256.977478,6.000848,5.888017,6.004581,0.0,...,0.0,0.922061,1.000000,25.000000,10.000000,000101,-70.117048,-70.50,2587.984375,2584.377197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149805,70.117048,168.00,264.279388,264.181641,264.485321,264.104553,3.836673,3.708954,3.456014,0.0,...,0.0,0.020916,0.007940,30.217773,31.002930,000000,70.117048,168.00,-40.105591,-40.208740
149806,70.117048,168.75,263.243958,263.457367,263.287903,263.835266,3.779803,3.540926,3.669104,0.0,...,0.0,0.025688,0.019084,19.599609,18.327148,000000,70.117048,168.75,48.488159,48.494385
149807,70.117048,170.25,266.316162,262.494171,262.690735,262.242371,5.866361,5.818576,6.012778,0.0,...,0.0,0.057624,0.039858,20.901367,20.796875,000000,70.117048,170.25,-41.070435,-41.771240
149808,70.117048,171.00,266.383087,264.283325,264.261658,264.064941,5.195374,5.332411,5.318110,0.0,...,0.0,0.384944,0.310225,28.710938,27.846680,000000,70.117048,171.00,498.417847,500.103760


In [72]:
#Save it
dfout.to_csv('tmp_data/all_yearly_points_with_oro.csv')
